In [6]:
import torch

train_loader = torch.load('train_loader.pth')

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GATRegressor(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GATRegressor, self).__init__()
        # Определение слоев
        self.gat = GATConv(input_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_attr):
        # Процесс вычислений через GAT слой с учётом весов рёбер
        x = self.gat(x, edge_index, edge_attr=edge_attr)
        x = F.relu(x)
        x = self.fc(x)  # Преобразование в конечный результат
        return x

In [8]:
COLUMNS = ['capacity', 'demand']

In [ ]:
from tqdm import tqdm

device = torch.device('cpu')

model = GATRegressor(input_dim=len(COLUMNS), hidden_dim=64, output_dim=1).to(device)  # output_dim=1 для регрессии
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
loss_fn = torch.nn.MSELoss()  # MSE для регрессии

def train(model, loader, optimizer, loss_fn, epochs=50):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in loader:
            batch = batch.to(device)
            
            # Маска для NaN в y
            mask = ~torch.isnan(batch.y)  # Пропускаем NaN значения

            if mask.sum() == 0:  # Если все значения NaN, пропускаем батч
                continue

            batch.x = batch.x[mask]  # Применяем маску к x
            batch.y = batch.y[mask]  # Применяем маску к y

            # Применяем маску для рёбер
            valid_nodes = torch.nonzero(mask).squeeze()  # Узлы, которые прошли фильтрацию
            node_map = {old_idx.item(): new_idx for new_idx, old_idx in enumerate(valid_nodes)}  # Отображение старых индексов на новые

            # Новый edge_index: фильтрация рёбер, где хотя бы один узел NaN
            edge_mask = mask[batch.edge_index[0]] & mask[batch.edge_index[1]]
            batch.edge_index = batch.edge_index[:, edge_mask]  # Пропускаем рёбра, где хотя бы один узел NaN

            # Переиндексация рёбер: заменяем старые индексы на новые
            # Проверяем, чтобы старые индексы были в словаре node_map
            batch.edge_index[0] = torch.tensor([node_map.get(idx.item(), -1) for idx in batch.edge_index[0]])
            batch.edge_index[1] = torch.tensor([node_map.get(idx.item(), -1) for idx in batch.edge_index[1]])

            # Убираем рёбра, которые указывают на неверные индексы
            valid_edges = (batch.edge_index[0] != -1) & (batch.edge_index[1] != -1)
            batch.edge_index = batch.edge_index[:, valid_edges]

            batch.edge_attr = batch.edge_attr[edge_mask] 

            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index, batch.edge_attr).squeeze()  # Прогоняем модель
            loss = loss_fn(out, batch.y.float())  # MSE требует float

            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(loader):.4f}")

train(model, train_loader, optimizer, loss_fn)

Epoch 1, Loss: 0.1354
Epoch 2, Loss: 0.1270
Epoch 3, Loss: 0.1203


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f162c53bd60>>
Traceback (most recent call last):
  File "/home/vasilstar/masterplanning/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


Epoch 4, Loss: 0.1168


In [5]:
torch.save(model.state_dict(), 'model.pth')